In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
#import os
#os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.regularizers import l1, l2
from keras.optimizers import SGD, RMSprop

In [2]:
import os
import pandas as pd
import numpy as np


from keras.preprocessing.image import ImageDataGenerator


In [3]:
root = 'leaf_classif'
print(os.path.join(root, 'train'))

leaf_classif\train


In [4]:
files = [file for file in os.listdir(root) if os.path.isfile(os.path.join(root, file))]
print(files)

[]


In [5]:
import random

main_dir = root  # Replace with your main directory

def label_from_folder_name(folder_name):
    if 'healthy' in folder_name:
        return 'healthy'  # Class label for healthy images
    elif 'diseased' in folder_name:
        return 'diseased'  # Class label for unhealthy images
    else:
        return None  # No specific label found
    
def custom_flow_from_directory(directory, target_size, batch_size):
    filenames = []
    labels = []
    i = 0

    for folder in os.listdir(directory):
        # first 2 folders
        if i > 1:
            break
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            for root, dirs, files in os.walk(folder_path):
                 for file in files:
                     filenames.append(os.path.join(root, file))
                     labels.append(label_from_folder_name(root))
        i = i + 1

    filenames = np.array(filenames)
    labels = np.array(labels, dtype=str)  # Ensure labels are strings

    return ImageDataGenerator(rescale=1./255).flow_from_dataframe(
        pd.DataFrame({"filename": filenames, "class": labels}),
        x_col="filename",
        y_col="class",
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary'
    )

# Use the custom function to load data
batch_size = 32
img_height, img_width = 150, 150

train_dir = os.path.join(main_dir, 'train')
test_dir = os.path.join(main_dir, 'test')
valid_dir = os.path.join(main_dir, 'valid')

train_generator = custom_flow_from_directory(train_dir, (img_height, img_width), batch_size)
test_generator = custom_flow_from_directory(test_dir, (img_height, img_width), batch_size)
valid_generator = custom_flow_from_directory(test_dir, (img_height, img_width), batch_size)


Found 412 validated image filenames belonging to 2 classes.
Found 10 validated image filenames belonging to 2 classes.
Found 10 validated image filenames belonging to 2 classes.


In [6]:
# Build a CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // batch_size
steps_per_epoch_valid = valid_generator.samples // batch_size
steps_per_epoch_test = test_generator.samples // batch_size

epochs = 10
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if valid_generator.samples % batch_size != 0:
    steps_per_epoch_valid += 1
if test_generator.samples % batch_size != 0:
    steps_per_epoch_test += 1

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/10
13/13 [==============================] - 54s 4s/step - loss: 0.8039 - accuracy: 0.5510 - val_loss: 0.6877 - val_accuracy: 0.5000
Epoch 2/10
13/13 [==============================] - 53s 4s/step - loss: 0.6577 - accuracy: 0.5995 - val_loss: 0.6691 - val_accuracy: 0.5000
Epoch 3/10
13/13 [==============================] - 53s 4s/step - loss: 0.6223 - accuracy: 0.6650 - val_loss: 0.6047 - val_accuracy: 0.5000
Epoch 4/10
13/13 [==============================] - 53s 4s/step - loss: 0.5294 - accuracy: 0.7354 - val_loss: 0.3769 - val_accuracy: 1.0000
Epoch 5/10
13/13 [==============================] - 53s 4s/step - loss: 0.3946 - accuracy: 0.8544 - val_loss: 0.2008 - val_accuracy: 1.0000
Epoch 6/10
13/13 [==============================] - 55s 4s/step - loss: 0.4182 - accuracy: 0.8180 - val_loss: 0.6544 - val_accuracy: 0.6000
Epoch 7/10
13/13 [==============================] - 54s 4s/step - loss: 0.4149 - accuracy: 0.8495 - val_loss: 0.2428 - val_accuracy: 1.0000
Epoch 8/10
13/13 [==

In [8]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=steps_per_epoch_test)

1/1 [==============================] - 1s 1s/step - loss: 0.1745 - accuracy: 1.0000


Fine tune CNN model - doubling number of epochs to 20, reducing steps size by half

In [9]:
# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // (batch_size * 2)
steps_per_epoch_valid = valid_generator.samples // (batch_size * 2)
steps_per_epoch_test = test_generator.samples // (batch_size * 2)

epochs = 20
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if valid_generator.samples % batch_size != 0:
    steps_per_epoch_valid += 1
if test_generator.samples % batch_size != 0:
    steps_per_epoch_test += 1

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/20
7/7 [==============================] - 30s 4s/step - loss: 0.2392 - accuracy: 0.9045 - val_loss: 0.1066 - val_accuracy: 1.0000
Epoch 2/20
7/7 [==============================] - 29s 4s/step - loss: 0.3075 - accuracy: 0.8591 - val_loss: 0.1340 - val_accuracy: 1.0000
Epoch 3/20
7/7 [==============================] - 29s 4s/step - loss: 0.2089 - accuracy: 0.9196 - val_loss: 0.1067 - val_accuracy: 1.0000
Epoch 4/20
7/7 [==============================] - 29s 4s/step - loss: 0.2416 - accuracy: 0.9152 - val_loss: 0.0919 - val_accuracy: 1.0000
Epoch 5/20
7/7 [==============================] - 30s 4s/step - loss: 0.1544 - accuracy: 0.9598 - val_loss: 0.1737 - val_accuracy: 1.0000
Epoch 6/20
7/7 [==============================] - 29s 4s/step - loss: 0.1392 - accuracy: 0.9636 - val_loss: 0.0781 - val_accuracy: 1.0000
Epoch 7/20
7/7 [==============================] - 29s 4s/step - loss: 0.1203 - accuracy: 0.9682 - val_loss: 0.1506 - val_accuracy: 1.0000
Epoch 8/20
7/7 [==================

In [10]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=steps_per_epoch_test)

1/1 [==============================] - 1s 1s/step - loss: 0.0412 - accuracy: 1.0000


Use a model built from a pretrained model

In [11]:
# Use pretrained model
from keras.applications import MobileNet
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# Define input image size expected by MobileNet
img_width, img_height = 224, 224

# Load the MobileNet model without the top classification layer
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification using sigmoid activation

# Combine base model with custom top layers
mobileModel = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
mobileModel.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [12]:
# Train the model

# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // batch_size
steps_per_epoch_valid = valid_generator.samples // batch_size
epochs = 10
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if valid_generator.samples % batch_size != 0:
    steps_per_epoch_valid += 1

model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/10
13/13 [==============================] - 54s 4s/step - loss: 0.0290 - accuracy: 0.9927 - val_loss: 0.0055 - val_accuracy: 1.0000
Epoch 2/10
13/13 [==============================] - 53s 4s/step - loss: 0.0259 - accuracy: 0.9951 - val_loss: 0.0572 - val_accuracy: 1.0000
Epoch 3/10
13/13 [==============================] - 53s 4s/step - loss: 0.0092 - accuracy: 1.0000 - val_loss: 0.0227 - val_accuracy: 1.0000
Epoch 4/10
13/13 [==============================] - 53s 4s/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 0.0323 - val_accuracy: 1.0000
Epoch 5/10
13/13 [==============================] - 53s 4s/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.0248 - val_accuracy: 1.0000
Epoch 6/10
13/13 [==============================] - 53s 4s/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0202 - val_accuracy: 1.0000
Epoch 7/10
13/13 [==============================] - 53s 4s/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0207 - val_accuracy: 1.0000
Epoch 8/10
13/13 [==

In [13]:
# Evaluate the model on the test dataset
loss, accuracy = mobileModel.evaluate(test_generator)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

1/1 [==============================] - 2s 2s/step - loss: 1.3134 - accuracy: 0.5000
Test Loss: 1.3134
Test Accuracy: 50.00%


In [14]:
# Fine tune the pretrained model

# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // (batch_size * 2)
steps_per_epoch_valid = valid_generator.samples // (batch_size * 2)
epochs = 20
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if valid_generator.samples % batch_size != 0:
    steps_per_epoch_valid += 1

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/20
7/7 [==============================] - 31s 4s/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0054 - val_accuracy: 1.0000
Epoch 2/20
7/7 [==============================] - 29s 4s/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0126 - val_accuracy: 1.0000
Epoch 3/20
7/7 [==============================] - 30s 4s/step - loss: 9.1172e-04 - accuracy: 1.0000 - val_loss: 0.0046 - val_accuracy: 1.0000
Epoch 4/20
7/7 [==============================] - 29s 4s/step - loss: 8.2134e-04 - accuracy: 1.0000 - val_loss: 0.0103 - val_accuracy: 1.0000
Epoch 5/20
7/7 [==============================] - 29s 4s/step - loss: 8.7130e-04 - accuracy: 1.0000 - val_loss: 0.0096 - val_accuracy: 1.0000
Epoch 6/20
7/7 [==============================] - 29s 4s/step - loss: 9.3733e-04 - accuracy: 1.0000 - val_loss: 0.0057 - val_accuracy: 1.0000
Epoch 7/20
7/7 [==============================] - 29s 4s/step - loss: 7.1712e-04 - accuracy: 1.0000 - val_loss: 0.0091 - val_accuracy: 1.0000
Epoch 8/20
7/7

In [15]:
# Evaluate the model on the test dataset
loss, accuracy = mobileModel.evaluate(test_generator)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")
\

1/1 [==============================] - 1s 1s/step - loss: 1.3134 - accuracy: 0.5000
Test Loss: 1.3134
Test Accuracy: 50.00%
